In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [2]:
def importrangeAllowAccessAutomation(get_spreadsheet_id, give_spreadsheet_id, creds):
    url = f'https://docs.google.com/spreadsheets/d/{get_spreadsheet_id}/externaldata/addimportrangepermissions?donorDocId={give_spreadsheet_id}&includes_info_params=true&cros_files=false'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {creds.token}',
    }
    # 데이터를 JSON 형태로 변환하고, POST 요청을 보냅니다.
    response = requests.post(url, headers=headers)
    # 응답 확인
    if 'application/json' in response.headers.get('Content-Type', ''):
        try:
            data = response.json()
            print(data)
        except ValueError:
            print("JSON 디코딩 실패", response.text)
    else:
        print("응답이 JSON 형식이 아닙니다:", response.text)

    

In [3]:
def get_existing_sheet_titles(service, spreadsheet_id):
    sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets = sheet_metadata.get('sheets', '')
    existing_titles = [sheet['properties']['title'] for sheet in sheets]
    return existing_titles

In [4]:
#송장 시트 생성 코드 
def create_sheets_with_formula(service, spreadsheet_id, start_number, new_sheet_number):
    existing_titles = get_existing_sheet_titles(service, spreadsheet_id)
    print(existing_titles)
    requests = []
    value_updates = []

    start_number = int(start_number)
    new_sheet_number = int(new_sheet_number)

    for i in range(0, new_sheet_number):
        sheet_number = start_number + i
        new_sheet_title = f"시트{sheet_number}"
        
        if new_sheet_title in existing_titles:
            print(f"Sheet '{new_sheet_title}' already exists. Skipping.")
            continue  # 이미 존재하는 시트 이름이면 건너뛴다

        formula = f"=iferror(FILTER('운송장배포'!A1:N, ('운송장배포'!J1:J={sheet_number})),\"\")"

        # 시트 추가 요청
        requests.append({
            'addSheet': {
                'properties': {
                    'title': new_sheet_title
                }
            }
        })
        
        range_value = f'{new_sheet_title}!A1'

        # 함수 적용을 위한 값 업데이트 준비
        value_updates.append({
            'range': range_value,
            'values': [[formula]]
        })

    # 모든 시트 추가
    if requests:
        service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body={'requests': requests}).execute()

    # 각 시트에 함수 적용
    for value_update in value_updates:
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=value_update['range'],
            valueInputOption="USER_ENTERED",
            body={'values': value_update['values']}
        ).execute()


In [5]:
#IMP {회원아이디}_발주서 생성코드
def create_sheets_without_formula(service, spreadsheet_id, start_number, new_sheet_number):
    existing_titles = get_existing_sheet_titles(service, spreadsheet_id)
    requests = []

    for i in range(new_sheet_number):
        sheet_number = start_number + i
        new_sheet_title = f"{sheet_number}_발주서"
        # formula = f'=IMPORTRANGE("1lkkPYvkEzPCoKhUf41U7hiosH3fCU4zzKo8JmqXLbTQ","\'발주서\'!$A$1:$N$999")'  # A2 셀에 적용할 수식
        if new_sheet_title in existing_titles:
            print(f"Sheet '{new_sheet_title}' already exists. Skipping.")
            continue  # 이미 존재하는 시트 이름이면 건너뛴다

        # 시트 추가 요청
        requests.append({
            'addSheet': {
                'properties': {
                    'title': new_sheet_title
                }
            }
        })
        
    if requests:
        service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body={'requests': requests}).execute()

In [6]:
def check_file_exists_in_folder(service, file_name, folder_id):
    """
    Google 드라이브의 특정 폴더에서 주어진 이름의 파일이 존재하는지 확인합니다.
    
    :param service: Google Drive API 서비스 인스턴스
    :param file_name: 확인할 파일의 이름
    :param folder_id: 검색할 폴더의 ID
    :return: 파일이 존재하면 True, 그렇지 않으면 False
    """
    query = f"name = '{file_name}' and '{folder_id}' in parents"
    response = service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
    files = response.get('files', [])
    return files


def apply_formula_to_cells(service, spreadsheet_id, sheet_title, new_daebong_spreadsheet_title):
    range_name = f"{sheet_title}!J11:J"  # J열 11행부터 끝까지   

    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", {new_daebong_spreadsheet_title})'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()

# 특정 셀에 수식을 적용하는 함수
def apply_formula_to_cell_sheet2(service, spreadsheet_id, sheet_title, new_daebong_spreadsheet_title, invoice_sheet_id):
    cell_range = 'A2'  # 수식을 적용할 셀 범위
    formula = f'=IMPORTRANGE("{invoice_sheet_id}","\'시트{new_daebong_spreadsheet_title}\'!$A$1:$N$999")'  # A2 셀에 적용할 수식
    range_name = f"{sheet_title}!{cell_range}"  # 지정된 시트의 특정 셀 범위
    value_range_body = {
        'values': [[formula]]  # 적용할 수식
    }
    
    request = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=range_name,
        valueInputOption='USER_ENTERED',
        body=value_range_body
    )
    request.execute()


# 특정 셀에 수식을 적용하는 함수
def apply_formula_to_cell_sheet3(service, spreadsheet_id, sheet_title, new_daebong_spreadsheet_title):
    print("시트 수정")

    cell_range_c1 = 'C1'  # 수식을 적용할 셀 범위
    range_name_c1 = f"{sheet_title}!{cell_range_c1}"  # 지정된 시트의 특정 셀 범위    


    cell_range_c2 = 'C2'  # 수식을 적용할 셀 범위
    range_name_c2 = f"{sheet_title}!{cell_range_c2}"  # 지정된 시트의 특정 셀 범위    


    cell_range_e3 = 'E3'  # 수식을 적용할 셀 범위
    range_name_e3 = f"{sheet_title}!{cell_range_e3}"  # 지정된 시트의 특정 셀 범위    

    cell_range_l8 = 'L8'  # 수식을 적용할 셀 범위
    range_name_l8 = f"{sheet_title}!{cell_range_l8}"  # 지정된 시트의 특정 셀 범위    

    print(range_name_c1, range_name_c2, range_name_e3, range_name_l8)

        # 셀에 적용할 텍스트와 스타일을 설정합니다.
    text_c1 = f"고객님의 발주서 번호는 {new_daebong_spreadsheet_title}번 입니다."
    text_c2 = f"입금 시 입금자명으로 {new_daebong_spreadsheet_title}을 입력해주세요."
    text_e3 = f"3. 입금자명을 {new_daebong_spreadsheet_title}로 합니다."
    text_l8 = f"{new_daebong_spreadsheet_title}"

    color_hex = "#2F55CB"

    
    blue = {"red": int(color_hex[1:3], 16) / 255.0, "green": int(color_hex[3:5], 16) / 255.0, "blue": int(color_hex[5:7], 16) / 255.0}  # 검은색
    black =  {"red": 0.0, "green": 0.0, "blue": 0.0}  # 검은색
    font_size_14 = 14  # 적용할 글자 크기
    font_size_12 = 12  # 적용할 글자 크기
    font_size_25 = 25  # 적용할 글자 크기
    bold = True  # 볼드 여부

    # 셀에 적용할 스타일을 설정합니다.
    text_format_c1 = {
        "foregroundColor": blue,
        "fontSize": font_size_14,
        "bold": bold
    }

    text_format_c2 = {
        "foregroundColor": blue,
        "fontSize": font_size_14,
        "bold": bold
    }

    text_format_e3 = {
        "foregroundColor": black,
        "fontSize": font_size_12,
        "bold": bold
    }

    text_format_l8 = {
        "foregroundColor": black,
        "fontSize": font_size_25,
        "bold": bold
    }

    spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')
    sheet_order_id = None
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "발주서":
            sheet_order_id = sheet.get('properties', {}).get('sheetId', '')


    request_1 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_c1}, "userEnteredFormat": {"textFormat": text_format_c1}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 0, 
                'startColumnIndex': 2,
                'endRowIndex':1,
                'endColumnIndex':3
            }
        }
    }

    request_2 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_c2}, "userEnteredFormat": {"textFormat": text_format_c2}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 1, 
                'startColumnIndex': 2, 
                'endRowIndex':2,
                'endColumnIndex':3
            }
        }
    }

    request_3 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_e3}, "userEnteredFormat": {"textFormat": text_format_e3}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 2, 
                'startColumnIndex': 4, 
                'endRowIndex':3,
                'endColumnIndex':5
            }
        }
    }


    request_4 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_l8}, "userEnteredFormat": {"textFormat": text_format_l8}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 7, 
                'startColumnIndex': 11, 
                'endRowIndex':8,
                'endColumnIndex':12
            }
        }
    }

    # # updateCells 요청을 사용하여 스프레드시트를 업데이트합니다.
    # response_1 = service.spreadsheets().updateCells(
    #     spreadsheetId=spreadsheet_id,
    #     body=request_1
    # ).execute()

    # response_2 = service.spreadsheets().updateCells(
    #     spreadsheetId=spreadsheet_id,
    #     body=request_2
    # ).execute()

    # response_3 = service.spreadsheets().updateCells(
    #     spreadsheetId=spreadsheet_id,
    #     body=request_3
    # ).execute()

    # response_4 = service.spreadsheets().updateCells(
    #     spreadsheetId=spreadsheet_id,
    #     body=request_4
    # ).execute()

    

    # 스프레드시트에 업데이트를 요청합니다.
    response_1 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_1]}
    ).execute()

    response_2 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_2]}
    ).execute()

    response_3 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_3]}
    ).execute()

    response_4 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_4]}
    ).execute()

    print("시트 수정 끝")

    print(response_1, response_2, response_3)



def apply_protect_data(service, spreadsheet_id):
    # 스프레드시트의 메타데이터에서 sheet_id 가져오기
    spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')

    sheet_order_id = None
    sheet_accumulated_order_id = None
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "발주서":
            sheet_order_id = sheet.get('properties', {}).get('sheetId', '')

        if sheet.get('properties', {}).get('title', '') == "누적발주":
            sheet_accumulated_order_id = sheet.get('properties', {}).get('sheetId', '')


    user_emails = ["daebong10x@gmail.com", "test-daebong@newdaebong.iam.gserviceaccount.com"]

    if sheet_order_id is not None:
        # 보호된 범위 설정
        requests = [
            {
                'addProtectedRange': {
                    'protectedRange': {
                        'range': {
                            'sheetId': sheet_order_id,
                            'startRowIndex': 10,  # 11행부터 시작 (0 기반 인덱스)
                            'startColumnIndex': 9,  # J열 (0 기반 인덱스)
                            # endRowIndex, endColumnIndex를 생략하면 범위의 나머지 부분을 포함합니다.
                        },
                        'description': 'Protected cells',
                        'editors': {
                            'users': user_emails,  # 이 셀을 편집할 수 있는 사용자
                        }
                    }
                }
            },
            {
                'addProtectedRange': {
                    'protectedRange': {
                        'range': {
                            'sheetId': sheet_accumulated_order_id,
                            'startRowIndex': 0,  # 11행부터 시작 (0 기반 인덱스)
                            'startColumnIndex': 0,  # J열 (0 기반 인덱스)
                            # endRowIndex, endColumnIndex를 생략하면 범위의 나머지 부분을 포함합니다.
                        },
                        'description': 'Protected cells',
                        'editors': {
                            'users': user_emails,  # 이 셀을 편집할 수 있는 사용자
                        }
                    }
                }
            }
        ]

        body = {
            'requests': requests
        }

        response = service.spreadsheets().batchUpdate(
            spreadsheetId=spreadsheet_id,
            body=body
        ).execute()

def copy_spreadsheet(service, file_id, start_number, new_sheet_number, service_sheets, folder_id, invoice_sheet_id):
    """
    Google 드라이브에 있는 스프레드시트의 복사본을 만들고 새로운 이름을 부여합니다.
    
    :param file_id: 복사할 스프레드시트의 파일 ID
    :param new_title: 복사본에 부여할 새로운 이름
    """
    spreadsheets_ids = []

    new_sheet_titles = []
    for i in range(start_number, start_number + new_sheet_number):
        new_title = f"{i}"
        # 폴더 내에 파일이 존재하지 않는 경우만 리스트에 추가
        if len(check_file_exists_in_folder(service, new_title, folder_id)) == 0:
            new_sheet_titles.append(new_title)

    for new_title in new_sheet_titles:
        copied_file = {'name': new_title}
        if len(check_file_exists_in_folder(service, new_title, folder_id)) > 0:
            continue
        else:
            try:
                # 스프레드시트 복사
                result = service.files().copy(fileId=file_id, body=copied_file).execute()
                new_spreadsheets_id = result['id']
                spreadsheets_ids.append(new_spreadsheets_id)
                
                apply_formula_to_cells(service_sheets, new_spreadsheets_id, '발주서', new_title)
                apply_formula_to_cell_sheet2(service_sheets, new_spreadsheets_id, '누적발주', new_title, invoice_sheet_id)
                apply_protect_data(service_sheets, new_spreadsheets_id)
                apply_formula_to_cell_sheet3(service_sheets, new_spreadsheets_id, '발주서', new_title)

                print(f"Spreadsheet '{new_title}' copied successfully. New spreadsheet ID: {result['id']}")
            except Exception as e:
                print(f"An error occurred while copying '{new_title}': {e}")

    print("new_sheet_titles", new_sheet_titles, spreadsheets_ids)

    output = []
    for i in range(0, len(new_sheet_titles)):
        element = [new_sheet_titles[i], spreadsheets_ids[i]]
        output.append(element)
    
    # input_data = [data for data in range(start_number, start_number + new_sheet_number + 1)]
    # if len(output) != len(input_data):
    #     included_numbers = [item[0] for item in output]
    #     for number in input_data:
    #         if number not in included_numbers:
    #             files_data = check_file_exists_in_folder(service, number, folder_id)
    #             output.append([files_data["name"], files_data["id"]])
    
    # print("output", output, input_data, included_numbers)
    # # output.sort(key=lambda x: x[0])
    # print(output)
    return output


In [7]:
def find_last_row(service, spreadsheet_id, column):
    """
    지정된 열에서 데이터가 있는 마지막 행의 인덱스를 반환합니다.
    """
    range_name = f'{column}:{column}'  # 예: 'D:D'
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id,
        range=range_name
    ).execute()
    values = result.get('values', [])

    if not values:
        return 1  # 데이터가 없으면 첫 번째 행 반환
    else:
        return len(values) + 1  # 데이터가 있는 마지막 행의 다음 행 반환



def append_to_sheet(service, spreadsheet_id, values):
    
    for title, new_spreadsheet_id in values:
        # D열의 마지막 행 찾기
        last_row_D = find_last_row(service, spreadsheet_id, 'D')
        range_name_D = f'시트1!D{last_row_D}'
        body_D = {'values': [[title]]}
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=range_name_D,
            valueInputOption='USER_ENTERED',
            body=body_D
        ).execute()

        # E열의 마지막 행 찾기
        last_row_E = find_last_row(service, spreadsheet_id, 'E')
        range_name_E = f'시트1!E{last_row_E}'
        body_E = {'values': [[new_spreadsheet_id]]}
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=range_name_E,
            valueInputOption='USER_ENTERED',
            body=body_E
        ).execute()


        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{new_spreadsheet_id}"

        last_row_F = find_last_row(service, spreadsheet_id, 'F')
        range_name_F = f'시트1!F{last_row_F}'
        body_F = {'values': [[spreadsheet_url]]}
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=range_name_F,
            valueInputOption='USER_ENTERED',
            body=body_F
        ).execute()

            


In [8]:
import time
def importrange_invoice_to_seller_ordersheet(service_sheets, invoice_sheet_id, TitleAndSpreadsheets_ids, creds):

    for sheet_title, spreadsheet_id in TitleAndSpreadsheets_ids:
        importrangeAllowAccessAutomation(spreadsheet_id, invoice_sheet_id, creds)



In [9]:
import time
def importrange_product_sheet(product_sheet_id, TitleAndSpreadsheets_ids, creds):

    for sheet_title, spreadsheet_id in TitleAndSpreadsheets_ids:
        importrangeAllowAccessAutomation(spreadsheet_id, product_sheet_id, creds)



In [10]:
def importrange_seller_to_imp(service_sheets, imp_sheet_id, TitleAndSpreadsheets_ids, creds):
    for sheet_title, spreadsheet_id in TitleAndSpreadsheets_ids:
        time.sleep(1)

        range_name = f'{sheet_title}_발주서!A2'
        formula = f'=IMPORTRANGE(\"{spreadsheet_id}\","\'발주서\'!$A$11:$M$99")'  # A2 셀에 적용할 수식
        value_range_body = {
            "range": range_name,
            "majorDimension": "ROWS",
            "values": [[formula]]
        }
        request = service_sheets.spreadsheets().values().update(
        spreadsheetId=imp_sheet_id, range=range_name,
        valueInputOption="USER_ENTERED", body=value_range_body)
        response = request.execute()

        print(f'Updated {sheet_title}!A2 with formula: {formula}')

        importrangeAllowAccessAutomation(imp_sheet_id, spreadsheet_id, creds)


In [12]:
# invoice_sheet_id = "1lkkPYvkEzPCoKhUf41U7hiosH3fCU4zzKo8JmqXLbTQ"
invoice_sheet_id = "1aFnJxj1MFHBZdwXyqbpJvKvemIs1NUW7KCbGPfl2l20"

# imp_sheet_id = "1nWRXCVmrdwaWE0oRqIHlArKwAPwL0pk5nNAjjN5raOo"
#imp_sheets_id 2
# imp_sheet_id = "1bb3MgGgyzSOhCxQ0LMVBVU5u5wjvMNmHTbFOjf1-9hs"
imp_sheet_id = "1PtUIKeUjbEVX8aY8xi2CcSPI1LCKFVrpqgsmXLlCr7w"
ordersheet_origin_id = "1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE"
signup_spreadsheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
folder_id = "16xozEJ0fwXmbnJORnRXfi5y_Gq3PwScH"
product_sheet_id = "1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4"

start_number = 584
new_sheet_number = 7

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 

    create_sheets_with_formula(service_sheets, invoice_sheet_id, start_number, new_sheet_number)
    create_sheets_without_formula(service_sheets, imp_sheet_id, start_number, new_sheet_number)
    output = copy_spreadsheet(service_drive, ordersheet_origin_id, start_number, new_sheet_number, service_sheets, folder_id, invoice_sheet_id)
    # # output = [['282', '1GUcwpeM7HFPZ_GDbH_3rQxUc-sIZGqm213YCrIxJXJg']]
    
    append_to_sheet(service_sheets, signup_spreadsheet_id, output)
    
    importrange_invoice_to_seller_ordersheet(service_sheets, invoice_sheet_id, output, creds)
    importrange_seller_to_imp(service_sheets, imp_sheet_id, output, creds)
    importrange_product_sheet(product_sheet_id, output, creds )
    


['운송장배포', '시트354', '시트355', '시트356', '시트357', '시트358', '시트359', '시트360', '시트361', '시트362', '시트363', '시트364', '시트365', '시트366', '시트367', '시트368', '시트369', '시트370', '시트371', '시트372', '시트373', '시트374', '시트375', '시트376', '시트377', '시트378', '시트379', '시트380', '시트381', '시트382', '시트383', '시트384', '시트385', '시트386', '시트387', '시트388', '시트389', '시트390', '시트391', '시트392', '시트393', '시트394', '시트395', '시트396', '시트397', '시트398', '시트399', '시트400', '시트401', '시트402', '시트403', '시트404', '시트405', '시트406', '시트407', '시트408', '시트409', '시트410', '시트411', '시트412', '시트413', '시트414', '시트415', '시트416', '시트417', '시트418', '시트419', '시트420', '시트421', '시트422', '시트423', '시트424', '시트425', '시트426', '시트427', '시트428', '시트429', '시트430', '시트431', '시트432', '시트433', '시트434', '시트435', '시트436', '시트437', '시트438', '시트439', '시트440', '시트441', '시트442', '시트443', '시트444', '시트445', '시트446', '시트447', '시트448', '시트449', '시트450', '시트451', '시트452', '시트453', '시트454', '시트455', '시트456', '시트457', '시트458', '시트459', '시트460', '시트461', '시트462', '시트463', 

In [11]:
#팜브릿지 발주서
# invoice_sheet_id = "1lkkPYvkEzPCoKhUf41U7hiosH3fCU4zzKo8JmqXLbTQ"
invoice_sheet_id = "1tw8Max2AQOcDYxlHcjxI2SHFba7tfhwtgXLN607FgB8"
# imp_sheet_id = "1nWRXCVmrdwaWE0oRqIHlArKwAPwL0pk5nNAjjN5raOo"
#imp_sheets_id 2
imp_sheet_id = "1B3J3ujJBH7URFV-UVdHr6T3J2MRRp6lODRSxa8700Cc"
ordersheet_origin_id = "1uJOuDW5nCTGivjkFs_Tr_hNbPV4OlJr79FocZqcMxPc"
signup_spreadsheet_id = "1LgUlxT2VggZHXRul12B8AJHvjUk6_xTF6aPVB0SzfEU"
folder_id = "1BT0v4uFTxmzBsvldAr9OlqERcGpgJX3u"

start_number = 68
new_sheet_number = 20

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 

    create_sheets_with_formula(service_sheets, invoice_sheet_id, start_number, new_sheet_number)
    create_sheets_without_formula(service_sheets, imp_sheet_id, start_number, new_sheet_number)
    output = copy_spreadsheet(service_drive, ordersheet_origin_id, start_number, new_sheet_number, service_sheets, folder_id, invoice_sheet_id)
    # output = [['282', '1GUcwpeM7HFPZ_GDbH_3rQxUc-sIZGqm213YCrIxJXJg']]

    # output = [['42','1-3v0_P9snEJDCIgf_bCrRqJaCdpeXpSfeDA5mRXTnMM'], ['43',  '1MTSDhdhkQDC-PlZylq-F2p1J9mfyhOh6cl1eB-xg4Uo'], ['44',  '1tNvFVs59CKnuxnuDoNeQTcar18x0g_WXfbPIuhhbxuA'], ['45','1lLQaweScUl26t49TGKDI2o9RTXH0lnp3CrG0juiwf6s'], ['46', '1AgH0HwvKFNrUAfswUpTzco8Kj4B7sHyn461ycgG-jPo',], ['47','1bNIXXgwc6vyZlaBgYSvdhZa4pxBLCEwpvj0wWTE67f4'], ['48', '1Pbj7ZavKg0HUO0yZaZqPCTnnpWG-r201A8OAM05myTk'], ['49', '123YJQ3rIpFcAY_oe6v9NqJotkfOtoiGakmItXbLJqYY'], ['50', '147gXc4DYJvLm0HJflQgDToWa7T2Mi7mMfHoDBot7JLY'], ['51', '1cUu7rd-BeC_HCAz8aVzcasT6Bo7GLt-eD2UNi__TvE8'],['52', '1Ou5UfwfkcQb4DBE7UB6Q7nxRTSWw_GdKGzMWaAb1NUQ'], ['53', '1-YHZtJG1EjY3FHx0_psAnVRO6fXCWFb26kh6sOuH75k'], ['54', '1SzVYi-x6upI27WTb9BSvvM5Y8asXlJT1fn_UTumz9cM'],  ['55', '1WuTJQfeYS7lT2nJbY4yUExTwp-v1cpJxABB5tD_orS8'], ['56', '1mQGa2pNTnj4pVtduFcSdSthv0iRRCklsBRe-PUxaskI'], ['57', '1-lDhal5NptWmReTjUVKIopwZav3dvxDdZuwKz4TNOQs'], ['58',   '1crRils7GOgjU7pOQjmSnQAx3s1GG9LNI7K7NzzMYQ0I'], ['59', '16HVBdr-cMzGbOOakhX5vEzAGfLzHwLarS07jmbzO5qQ'], ['60', '1HsL7BWQiVwj_abUUcssc9gVC980H-eLjlAfmog9c96w'], ['61', '1mu9QOlMhv3amHC2ZWKQ15c06s_S_Epit2H7a1tAdWcs'] ]
    
    append_to_sheet(service_sheets, signup_spreadsheet_id, output)
    
    importrange_invoice_to_seller_ordersheet(service_sheets, invoice_sheet_id, output, creds)
    importrange_seller_to_imp(service_sheets, imp_sheet_id, output, creds)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=520657704422-o724cfbjdm5vnt6iko73r4buo8gjr68k.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fscript.projects+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&state=maLMqd5Ynj0ol3u4IGC1okN5USudVV&access_type=offline
['운송장배포', '당일발주내역', '당일송장번호', '미출고송장', '미출고내역', '1월', '시트19', '시트20', '시트21', '시트22', '시트23', '시트24', '시트25', '시트26', '시트27', '시트28', '시트29', '시트30', '시트31', '시트32', '시트33', '시트34', '시트35', '시트36', '시트37', '시트38', '시트39', '시트40', '시트41', '시트42', '시트43', '시트44', '시트45', '시트46', '시트47', '시트48', '시트49', '시트50', '시트51', '시트52', '시트53', '시트54', '시트55', '시트56', '시트57', '시트58', '시트59', '시트60', '시트61']
시트 수정
발주서!C1 발주서!C2 발주서!E3 발주서!L8
시트 수정 끝
{'spreadsheetId': '1IkAMjSV7d95MmCXiGPRS

In [ ]:
tests = ['441', '442', '443', '444', '445', '446', '447', '448', '449', '450', '451', '452', '453', '454', '455', '456', '457', '458', '459', '460', '461', '462', '463', '464', '465', '466', '467', '468', '469', '470'] 
ids = ['1OQgizUzoRSNpzjUA6qvnYvyIMyadVQsTYfFxzT4cfrE', '1E2i1IJRuPAoDmXum5vjrNo65mmaEjIyBKhENEH7VZEE', '1IOCubZrxV-dR1uNCFSgP3juhLIlMIMHYVaJACAMu1tY', '1cGSDZu2qK_Cx6jXa_qzlnX1fumXlyIUi2LIKVDX70Sw', '1W52kNbCaDXafLTYxMfJ3pnMmzU8o1UIkYonsrmm2V1k', '1FAInvjJCqdCk1ZyoFWv-kUhF6pBmZpN43AK_z_Exkjs', '1CwPGQ9T353ee1YZRy3HVB1KnF2ay0cZkA5LLUXpXrrU', '1_LMtNqEmzXV9Fm_3X7N6C5xgTjKNsHrH8tO_4zyK7yU', '1ug6Cq86Uyqd1GaUMp-SLiLhR9Bl37QIU0Dp-JvXVFMU', '1zjp2aZRe5kOnnV1V5zpuEq9KASE3zSpmKr9_hLBl-MA', '1hVWqPBOI7pvoYteC2ts7XeKisIfTEDIbRIPtbOYA9MQ', '1v0MX1YZ-BMTKdylF8reNRAf-9zzL5RqrERrhasZaTLQ', '1uXcc-VYZFV2qLwocgAlY9XTzXCP_-0gkGQe5YuUAqN8', '1hp4jKgpr3wvpboEaN-eiDMlTusLa1oZxf-08c8Soo40', '1qIK7MsVk0bB-zt7_OkfMP4Tu4xbh77hmlMdS_dLQOSA', '1oOWKbvHYbE_XH5OQSSzuWxlvNaIcnlcMBq-aZbabsb0', '1pL_nAvqcZMgXnT3xtJu0hdqFvzmm-gobaHJSzStDY2Y', '11D86Ux0Bp4WlCWXwAGzESQUcHuPVHPzJV_ZKwYhc1vE', '1-shYws5SdWblFyiq9IRlxNAwwFyyjb6VvDzqPFAuaDo', '1concNI0rwdwObLN-lnm-HAw1JGU0BgTwuLD4yMndp1Q', '104ZWc3RoB0WjkITYQLyFBLvkZ5UywvIJZHoWIha9iTk', '1g1Wt26k_G9i6gx3j2B-ogZ7O2VVeJRv6OjXjKmPQmfc', '1LjeX4IP8i13tKX_HqQgCvxM3k45QpdGluQtnQiOOaOQ', '1JG7u3zTmaH_WfKri2MvNGfUwGua3WAaCj-fXW6vSW2w', '1P6p_fSZ-E28ThAwYOzanRXHKDuyPyVqLDTtyexlI3EY', '1qCEZqPwP8x5FNOU038iSwx4M_cP_uP9J1Yhuv7qFk-0', '1N8OZ2HkeX7woyNezM9OP4hQtZmhhj7YuDhWrjW645JQ', '1dMLFlUP-oXYHnmY-Qm4hK0QUykH6sZyPdPza35HPWwU', '1l_dyacnX7lhxtPot1czzBiWEnLODcVuEJ77vVddfDdw', '1oQvBGq3yKYDN9GxaK95bmVO99aVwDn9KGxJ_ceI2BFo']


print(len(tests))
print(len(ids))

data = []

for i in range(len(tests)):
    data.append([tests[i], ids[i]])

print(data)

In [20]:
TARGET_SPREADSHEET_ID = '1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU'
SHEET_NAME = '시트1'

rows = []
if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 

    range_name = f'{SHEET_NAME}!D2:E'
    result = service_sheets.spreadsheets().values().get(
        spreadsheetId=TARGET_SPREADSHEET_ID, range=range_name).execute()
    rows = result.get('values', [])


    # for row in rows:
    #     source_spreadsheet_id = row[1]  # 스프레드시트 ID
    #     # 발주서 시트의 데이터를 가져옵니다.
    #     order_sheet_range = f'{ORDER_SHEET_PREFIX}!A:M'
    #     order_data = service.spreadsheets().values().get(
    #         spreadsheetId=source_spreadsheet_id, range=order_sheet_range).execute()
    #     order_rows = order_data.get('values', [])


In [26]:
print(rows)


def apply_protect(service, spreadsheet_id):
    # 스프레드시트의 메타데이터에서 sheet_id 가져오기
    spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')

    sheet_order_id = None
    sheet_accumulated_order_id = None
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "발주서":
            sheet_order_id = sheet.get('properties', {}).get('sheetId', '')

        if sheet.get('properties', {}).get('title', '') == "누적발주":
            sheet_accumulated_order_id = sheet.get('properties', {}).get('sheetId', '')


    user_emails = ["daebong10x@gmail.com", "test-daebong@newdaebong.iam.gserviceaccount.com"]

    if sheet_order_id is not None:
        # 보호된 범위 설정
        requests = [
            {
                'addProtectedRange': {
                    'protectedRange': {
                        'range': {
                            'sheetId': sheet_accumulated_order_id,
                            'startRowIndex': 0,  # 11행부터 시작 (0 기반 인덱스)
                            'startColumnIndex': 0,  # J열 (0 기반 인덱스)
                            # endRowIndex, endColumnIndex를 생략하면 범위의 나머지 부분을 포함합니다.
                        },
                        'description': 'Protected cells',
                        'editors': {
                            'users': user_emails,  # 이 셀을 편집할 수 있는 사용자
                        }
                    }
                }
            }
        ]

        body = {
            'requests': requests
        }

        response = service.spreadsheets().batchUpdate(
            spreadsheetId=spreadsheet_id,
            body=body
        ).execute()

[['226', '1-J6V0nLHQlxLA8PbTiTXaGt08VlqGv5fo3kfaH35FJE'], ['227', '14KS38kY_hZD2061EmIaAm3cA3Rw9R2mEDoIczxdk3sQ'], ['228', '1hrmwW1aznOQJewXDlyCsfCqEvsTjh_BXXVSZAqwH2r0'], ['229', '1aEZw4wsj1HXCuxvcc5mnvJ0UI8Qkuo05-zmYPzblSWc'], ['230', '1OxaNX2XUK5CTsqseH6Dnj_3Bhi1FNn_79YAAH1sOdhE'], ['231', '1CC5HAoulfHpomxSJWU2Zr0doNLwZQYj9uMsnk9YlUIY'], ['232', '1iKJnf9JCFOvBUqcufaX9xs2YVrqIpVGTxFruIP6wwhg'], ['233', '1911UP9nR65fxxJO_Na8886T-Q-FBPvMaRs-twm4KsoI'], ['234', '13gTVm1GQEnrQqgc_Q55M8cWyrWrUAmt2gt_v7Ltm-18'], ['235', '1wKg9_xHUVFRujWUsX9unEN38pD8yg3B_c1f8z1C8FI4'], ['236', '102YTIjanaJqeckd-zJ_s5d-EgUeZQUB1F7G9wWw_-6w'], ['237', '1P3Z7nF8kRh0H84yk26Y3G4EzfY_AWRPdU4mNyppTNaM'], ['238', '1ywXZQV6GXPP-xDAfAJ1Q0AqBD4TTTV214qOvUF79_aI'], ['239', '1hAdI2qGPe1UIV-8qPLBGOk2XLvSwub0LSEQBJSL4XXE'], ['240', '1wvqrK1DUrey1EVr3vOY5KH3vngeZHyT_aZKf1a7RAig'], ['241', '1gADqw_nKmZGBWvkRGB1YbfxMdiDpCGGFX-KMNyLVlO4'], ['242', '1ghDdb9oxfzMtlFcf0UJ107tw53cS54xaYcyMKLBMAlY'], ['243', '1Zt4SVQIelLhPu-x24A1S

In [27]:
import time


invoice_sheet_id = "1lkkPYvkEzPCoKhUf41U7hiosH3fCU4zzKo8JmqXLbTQ"
# imp_sheet_id = "1nWRXCVmrdwaWE0oRqIHlArKwAPwL0pk5nNAjjN5raOo"
#imp_sheets_id 2
imp_sheet_id = "1bb3MgGgyzSOhCxQ0LMVBVU5u5wjvMNmHTbFOjf1-9hs"
ordersheet_origin_id = "1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE"
signup_spreadsheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"

start_number = 281
new_sheet_number = 1

if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 


    for sheet_title, spreadsheet_id in rows:
        time.sleep(1)
        
        cell_range = 'A2'  # 수식을 적용할 셀 범위

        formula = f'=IMPORTRANGE(\"{invoice_sheet_id}\","\'시트{sheet_title}\'!$A$1:$N$999")'  # A2 셀에 적용할 수식
        range_name = f"누적발주!{cell_range}"  # 지정된 시트의 특정 셀 범위
        value_range_body = {
            'values': [[formula]]  # 적용할 수식
        }
        
        request = service_sheets.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=range_name,
            valueInputOption='USER_ENTERED',
            body=value_range_body
        )
        request.execute()

        apply_protect(service_sheets, spreadsheet_id)
        

        importrangeAllowAccessAutomation(spreadsheet_id, invoice_sheet_id, creds)


        # sheet_metadata = service_sheets.spreadsheets().get(spreadsheetId=imp_sheet_id).execute()
        # sheets = sheet_metadata.get('sheets', '')

    # for i in range(0, new_sheet_number):
    #     range = f'{start_number + i}_발주서!A2'
    #     formula = f'=IMPORTRANGE(\"{TitleAndSpreadsheets_ids[i]}\","\'발주서\'!$A$1:$N$999")'  # A2 셀에 적용할 수식
    #     value_range_body = {
    #         "range": range,
    #         "majorDimension": "ROWS",
    #         "values": [[formula]]
    #     }
    #     request = service_sheets.spreadsheets().values().update(
    #     spreadsheetId=imp_sheet_id, range=range,
    #     valueInputOption="USER_ENTERED", body=value_range_body)
    #     response = request.execute()

    #     print(f'Updated {start_number + i}!A2 with formula: {formula}')

    #     importrangeAllowAccessAutomation(imp_sheet_id, TitleAndSpreadsheets_ids[i], creds)


            

JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]}'
{"success":true}
JSON 디코딩 실패 )]

In [31]:
creds = oauthByWeb()
service_sheets = build('sheets', 'v4', credentials=creds)
# sheet_metadata = service_sheets.spreadsheets().get(spreadsheetId=imp_sheet_id).execute()
# sheets = sheet_metadata.get('sheets', '')


for sheet_title, spreadsheet_id in rows:
    time.sleep(1)

    range_name = f'{sheet_title}_발주서!A2'
    formula = f'=IMPORTRANGE(\"{spreadsheet_id}\","\'발주서\'!$A$11:$M$99")'  # A2 셀에 적용할 수식
    value_range_body = {
        "range": range_name,
        "majorDimension": "ROWS",
        "values": [[formula]]
    }
    request = service_sheets.spreadsheets().values().update(
    spreadsheetId=imp_sheet_id, range=range_name,
    valueInputOption="USER_ENTERED", body=value_range_body)
    response = request.execute()

    print(f'Updated {sheet_title}!A2 with formula: {formula}')

    importrangeAllowAccessAutomation(imp_sheet_id, spreadsheet_id, creds)

Updated 226!A2 with formula: =IMPORTRANGE("1-J6V0nLHQlxLA8PbTiTXaGt08VlqGv5fo3kfaH35FJE","'발주서'!$A$11:$M$99")
JSON 디코딩 실패 )]}'
{"success":true}
Updated 227!A2 with formula: =IMPORTRANGE("14KS38kY_hZD2061EmIaAm3cA3Rw9R2mEDoIczxdk3sQ","'발주서'!$A$11:$M$99")
JSON 디코딩 실패 )]}'
{"success":true}
Updated 228!A2 with formula: =IMPORTRANGE("1hrmwW1aznOQJewXDlyCsfCqEvsTjh_BXXVSZAqwH2r0","'발주서'!$A$11:$M$99")
JSON 디코딩 실패 )]}'
{"success":true}
Updated 229!A2 with formula: =IMPORTRANGE("1aEZw4wsj1HXCuxvcc5mnvJ0UI8Qkuo05-zmYPzblSWc","'발주서'!$A$11:$M$99")
JSON 디코딩 실패 )]}'
{"success":true}
Updated 230!A2 with formula: =IMPORTRANGE("1OxaNX2XUK5CTsqseH6Dnj_3Bhi1FNn_79YAAH1sOdhE","'발주서'!$A$11:$M$99")
JSON 디코딩 실패 )]}'
{"success":true}
Updated 231!A2 with formula: =IMPORTRANGE("1CC5HAoulfHpomxSJWU2Zr0doNLwZQYj9uMsnk9YlUIY","'발주서'!$A$11:$M$99")
JSON 디코딩 실패 )]}'
{"success":true}
Updated 232!A2 with formula: =IMPORTRANGE("1iKJnf9JCFOvBUqcufaX9xs2YVrqIpVGTxFruIP6wwhg","'발주서'!$A$11:$M$99")
JSON 디코딩 실패 )]}'
{"success

In [ ]:
def add_permission(service, file_id, user_email, role='owner', type='user'):
    """
    특정 파일에 대한 권한을 부여합니다.

    :param file_id: 권한을 부여할 파일의 ID
    :param user_email: 권한을 부여할 사용자의 이메일 주소
    :param role: 부여할 권한의 종류 ('owner', 'writer', 'commenter', 'reader' 중 하나)
    :param type: 권한을 부여할 대상의 유형 ('user', 'group', 'domain', 'anyone' 중 하나)
    """
    permission = {
        'type': type,
        'role': role,
        'emailAddress': user_email
    }
    try:
        # 파일에 권한을 추가합니다.
        service.permissions().create(
            fileId=file_id,
            body=permission,
            transferOwnership=True, 
            fields='id'
        ).execute()
        print(f"Permission added for {user_email}")
    except Exception as e:
        print(f"An error occurred: {e}")


def delete_spreadsheet(service, file_id):
    try:
        service.files().delete(fileId=file_id).execute()
        print(f'File with ID {file_id} was deleted.')
    except Exception as e:
        print(f'An error occurred: {e}')

In [27]:
# Spreadsheet '복사본_900_발주서' copied successfully. New spreadsheet ID: 1Cit4dQrybsgDtizTp3o66SnhGQsDr8wN-0TfQhNHTAw
add_permission(service_drive, '1Cit4dQrybsgDtizTp3o66SnhGQsDr8wN-0TfQhNHTAw', 'daebong10x@gmail.com')


Permission added for daebong10x@gmail.com


In [30]:
delete_spreadsheet(service_drive, "1Cit4dQrybsgDtizTp3o66SnhGQsDr8wN-0TfQhNHTAw");

File with ID 1Cit4dQrybsgDtizTp3o66SnhGQsDr8wN-0TfQhNHTAw was deleted.


In [69]:
import requests

# Zapier Webhook URL

def zapierWebhookCreate(processed_data):
    webhook_url = 'https://hooks.zapier.com/hooks/catch/17482879/30lgxcd/'

    # 전송할 데이터
    data = {
        'user_data': {
            'name': processed_data[0],
            'phone': processed_data[1],
            'description':processed_data[2],
        }
    }

    # POST 요청으로 데이터 전송
    response = requests.post(webhook_url, json=data)

    # 응답 출력
    print(response.text)

In [77]:
SPREADSHEET_ID = '1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU'
processed_data = []
if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)

    RANGE_NAME = '셀러컨택포인트!J56:J110'  # J열 데이터 범위
    B_C__J_RANGE = '셀러컨택포인트!B56:L110'  # B와 C열 데이터 범위
    # L_RANGE = '셀러컨택포인트!L2:L247'  # L열 데이터 범위

   # 스프레드시트에서 J열 데이터 읽기
    sheet = service_sheets.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    j_values = result.get('values', [])
    print(j_values)
    print(len(j_values))
    # print(len(j_values))

    # 스프레드시트에서 B와 C열 데이터 읽기
    bc_result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=B_C__J_RANGE).execute()
    bc_values = bc_result.get('values', [])
    print(bc_values)
    print(len(bc_values)) 

    # l_result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=L_RANGE).execute()
    # l_values = l_result.get('values', [])

    # print(l_values)
    # print(len(l_values))

    

    if not j_values:
            print('No data found in J column.')
    else:
        for index, row in enumerate(j_values):
            if row:  # J열에 데이터가 있는 경우만 처리
                try:
                    # print(row)
                    # B열과 C열 데이터 가져오기
                    bc_data = bc_values[index]
                    name = f"{bc_data[0]}_{bc_data[1]}"
                    # print(name)
                    phone = row[0]
                    description = bc_data[-1]
                    processed_data.append([name, phone, description])

                    print("name", "phone", "description")
                    print(name, phone, description)
                except IndexError:
                    # B와 C열 데이터가 없는 경우를 처리
                    continue

        print(processed_data)


[[], [], ['01032963537'], [], [], [], [], [], ['01056922068'], [], [], [], [], ['01056353212'], [], ['01033692354'], [], [], ['01024206566'], [], [], ['01033246299'], [], ['01029676416'], [], [], [], [], [], ['01023949182'], ['01073087298'], [], [], [], [], [], [], [], ['01090879535'], [], [], [], [], [], [], [], [], [], ['01026959800'], ['01097990710']]
50
[['76', '안서현', '76_안서현', '팀채팅임', '', '', '', '', '', '1RTS4gFbGpELMAoXn6gEq30U8Gxsw9sc759XAeLIalIo', 'https://docs.google.com/spreadsheets/d/1RTS4gFbGpELMAoXn6gEq30U8Gxsw9sc759XAeLIalIo'], ['77', '계절의과일', '77_계절의과일', 'O', '', '', '', '', '', '1UfwIKFVNNRT8VjN-hP_YWirS04TguC6HQqHTAYx770k', 'https://docs.google.com/spreadsheets/d/1UfwIKFVNNRT8VjN-hP_YWirS04TguC6HQqHTAYx770k'], ['78', '살가운청과', '78_살가운청과', 'O', '', '', '', '', '01032963537', '1gOsaEoEtS8z7HTiyi4GHWiPyt1SwxNrd55AzzcD02x4', 'https://docs.google.com/spreadsheets/d/1gOsaEoEtS8z7HTiyi4GHWiPyt1SwxNrd55AzzcD02x4'], ['79', '달곰한', '79_달곰한', '팀채팅임', '', '', '', '', '', '1-BzndUe5

In [78]:
print(len(processed_data))
# zapierWebhookCreate(processed_data[0])

12


In [79]:
import time

for i in processed_data:
    time.sleep(1)
    zapierWebhookCreate(i)

{"attempt":"018e430c-1b98-783d-d54a-090783a45ec2","id":"018e430c-1b98-783d-d54a-090783a45ec2","request_id":"018e430c-1b98-783d-d54a-090783a45ec2","status":"success"}

{"attempt":"018e430c-2291-2c08-be59-e513454509aa","id":"018e430c-2291-2c08-be59-e513454509aa","request_id":"018e430c-2291-2c08-be59-e513454509aa","status":"success"}

{"attempt":"018e430c-27cc-d889-c781-d7c6896f9ac9","id":"018e430c-27cc-d889-c781-d7c6896f9ac9","request_id":"018e430c-27cc-d889-c781-d7c6896f9ac9","status":"success"}

{"attempt":"018e430c-2e4f-8cb2-e4cf-1f9a4df9ad88","id":"018e430c-2e4f-8cb2-e4cf-1f9a4df9ad88","request_id":"018e430c-2e4f-8cb2-e4cf-1f9a4df9ad88","status":"success"}

{"attempt":"018e430c-333d-6233-d2c5-298e188659cf","id":"018e430c-333d-6233-d2c5-298e188659cf","request_id":"018e430c-333d-6233-d2c5-298e188659cf","status":"success"}

{"attempt":"018e430c-382d-165c-4cfa-8083dabd70ef","id":"018e430c-382d-165c-4cfa-8083dabd70ef","request_id":"018e430c-382d-165c-4cfa-8083dabd70ef","status":"success"}

In [38]:
SPREADSHEET_ID = '1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU'
processed_data_sheet_1 = []
if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)

    RANGE_NAME = '시트1!B2:B42'  # J열 데이터 범위
    B_C_RANGE = '시트1!A2:D42'  # B와 C열 데이터 범위

   # 스프레드시트에서 J열 데이터 읽기
    sheet = service_sheets.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    j_values = result.get('values', [])
    print(j_values)
    print(len(j_values))

    # 스프레드시트에서 B와 C열 데이터 읽기
    bc_result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=B_C_RANGE).execute()
    bc_values = bc_result.get('values', [])

    

    if not j_values:
            print('No data found in J column.')
    else:
        for index, row in enumerate(j_values):
            # print(row)
            try:
                # B열과 C열 데이터 가져오기
                bc_data = bc_values[index]
                # print(bc_data)
                # print(bc_data[3])
                # print(bc_data[0])
                name = f"{bc_data[3]}_{bc_data[0]}"
                phone = bc_data[1]
                # print(name, phone)
                processed_data_sheet_1.append([name, phone])
            except IndexError:
                # B와 C열 데이터가 없는 경우를 처리
                continue

print(processed_data_sheet_1)


[['1036445918'], ['1062244640'], ['1088983526'], ['1036514834'], ['1076269266'], ['1053101615'], ['1095943574'], ['1094679162'], ['1083634871'], ['1056498626'], ['1088428798'], ['1031179717'], ['1094407770'], ['1098609168'], ['1038259851'], ['1062755150'], ['1025402281'], ['1065288131'], ['1035538459'], ['1080791101'], ['1090113286'], ['1083227287'], ['1089098189'], ['1024744462'], ['1032076016'], ['1084111694'], ['1059045282'], ['1050596491'], ['1036285480'], ['1022956588'], ['1038681643'], ['1036959108'], ['1025639544'], ['1057802238'], ['1063312935'], ['1041590971'], ['1031410087'], ['1036959108'], ['1026344132'], ['1071607103'], ['1033897854']]
41
[['226_농부의마음', '1036445918'], ['227_새콤달콤써니네', '1062244640'], ['228_예스마켓', '1088983526'], ['229_디제이 유통', '1036514834'], ['230_문제우', '1076269266'], ['231_다마라', '1053101615'], ['232_이름 : 이미경 회사명 : 글리밍 발리', '1095943574'], ['233_메론메롱', '1094679162'], ['234_박인애', '1083634871'], ['235_정이나', '1056498626'], ['236_해룡산업', '1088428798'], ['237_나는된다GO

In [42]:
import time

print(len(processed_data_sheet_1))

for i in processed_data_sheet_1:
    time.sleep(1)
    zapierWebhookCreate(i)

41
{"attempt":"018e40cd-74cc-a1da-fcaf-5a9d8b6d96db","id":"018e40cd-74cc-a1da-fcaf-5a9d8b6d96db","request_id":"018e40cd-74cc-a1da-fcaf-5a9d8b6d96db","status":"success"}

{"attempt":"018e40cd-79fb-49cd-5ae8-3238570aca6a","id":"018e40cd-79fb-49cd-5ae8-3238570aca6a","request_id":"018e40cd-79fb-49cd-5ae8-3238570aca6a","status":"success"}

{"attempt":"018e40cd-7eec-1867-f3d5-0c23025ca317","id":"018e40cd-7eec-1867-f3d5-0c23025ca317","request_id":"018e40cd-7eec-1867-f3d5-0c23025ca317","status":"success"}

{"attempt":"018e40cd-8513-5a1d-afc1-c77951f6bad5","id":"018e40cd-8513-5a1d-afc1-c77951f6bad5","request_id":"018e40cd-8513-5a1d-afc1-c77951f6bad5","status":"success"}

{"attempt":"018e40cd-89fe-e77e-58b9-025ea67816ca","id":"018e40cd-89fe-e77e-58b9-025ea67816ca","request_id":"018e40cd-89fe-e77e-58b9-025ea67816ca","status":"success"}

{"attempt":"018e40cd-8ff4-e9b0-8b9a-9a9549b0a22d","id":"018e40cd-8ff4-e9b0-8b9a-9a9549b0a22d","request_id":"018e40cd-8ff4-e9b0-8b9a-9a9549b0a22d","status":"succes